# Carga da Dimensão Cliente

Este notebook realiza a carga da dimensão cliente (dim_cliente) a partir dos dados da tabela raw de cadastro de cliente.

## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_timestamp, sha2, coalesce
from delta.tables import DeltaTable

## Start Spark Session

In [2]:
# Inicializa uma SparkSession com metastore persistente
builder = (
    SparkSession.builder
    .appName("Carga Dimensão Cliente")
    .master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.warehouse.dir", "D:/Projetos/DataLake/spark-warehouse")
    .config(
        "javax.jdo.option.ConnectionURL",
        "jdbc:derby:;databaseName=D:/Projetos/DataLake/metastore_db;create=true"
    )
    .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado com metastore persistente!\n")


✅ Spark 3.5.7 iniciado com metastore persistente!



## Define Delta Table Paths

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_cadcliente = f"{base_bronze_path}/raw_cadcliente"
delta_path_dim_cliente = f"{base_gold_path}/dim_cliente"

## Read Source Data

In [4]:
# Lê a tabela bronze de cadastro de clientes
df_raw_cadcliente = spark.read.format("delta").load(delta_path_cadcliente)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_cadcliente}")
print(f"Total de registros: {df_raw_cadcliente.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cadcliente
Total de registros: 2


## Transform Data

In [5]:
# Gera chave surrogate e transforma os dados para dimensão cliente
df_dim_cliente = df_raw_cadcliente.select(
    # Gera surrogate key usando sha2 do CPF
    sha2(col("cpf"), 256).alias("sk_cliente"),
    col("cpf").alias("cd_cpf_pessoa"),
    col("nome").alias("nm_cliente"),
    col("data_nascimento").cast("date").alias("dt_nascimento"),
    
    col("email").alias("ds_email"),
    col("telefone").alias("nr_telefone")
)

# Remove duplicatas baseado no CPF (mantém o registro mais recente)
df_dim_cliente = df_dim_cliente.dropDuplicates(["cd_cpf_pessoa"])

## Write Data

In [6]:
# Carrega a tabela Delta existente
deltaTable = DeltaTable.forPath(spark, delta_path_dim_cliente)

# Realiza o merge (upsert) na tabela Delta
(
    deltaTable.alias("target")
    .merge(
        df_dim_cliente.alias("source"),
        "target.cd_cpf_pessoa = source.cd_cpf_pessoa"  # Merge por CPF (chave natural)
    )
    # Quando o registro já existe e há diferença nos campos relevantes → atualiza
    .whenMatchedUpdate(
        condition="""
            coalesce(target.nm_cliente, '') != coalesce(source.nm_cliente, '') OR 
            coalesce(target.dt_nascimento, '') != coalesce(source.dt_nascimento, '') OR 
            coalesce(target.ds_email, '') != coalesce(source.ds_email, '') OR 
            coalesce(target.nr_telefone, '') != coalesce(source.nr_telefone, '')
        """,
        set={
            "nm_cliente": "source.nm_cliente",
            "dt_nascimento": "source.dt_nascimento",
            "ds_email": "source.ds_email",
            "nr_telefone": "source.nr_telefone",
            "ts_atualizacao": "current_timestamp()"  # Atualiza timestamp no update
        }
    )
    # Quando o registro não existe → insere novo
    .whenNotMatchedInsert(values={
        "sk_cliente": "source.sk_cliente",
        "cd_cpf_pessoa": "source.cd_cpf_pessoa",
        "nm_cliente": "source.nm_cliente",
        "dt_nascimento": "source.dt_nascimento",
        "ds_email": "source.ds_email",
        "nr_telefone": "source.nr_telefone",
        "ts_insercao": "current_timestamp()",  # Data/hora da inserção
        "ts_atualizacao": "null"               # Ainda não foi atualizado
    })
    .execute()
)

# 📊 Obtém métricas do último merge
last_operation = deltaTable.history(1).select("operationMetrics").collect()[0][0]
inserted = int(last_operation.get("numTargetRowsInserted", 0))
updated = int(last_operation.get("numTargetRowsUpdated", 0))

print(f"✅ Merge realizado com sucesso")
print(f"🆕 Inseridos: {inserted}")
print(f"♻️ Atualizados: {updated}")

✅ Merge realizado com sucesso
🆕 Inseridos: 2
♻️ Atualizados: 0


## Stop Spark Session

In [7]:
# Encerra a SparkSession
spark.stop()